# Create a transpiler plugin

Creating a [transpiler plugin](transpiler-plugins) is a great way to share your transpilation code with the wider Qiskit community, allowing other users to benefit from the functionality you've developed. Thank you for your interest in contributing to the Qiskit community!

Before you create a transpiler plugin, you need to decide what kind of plugin is appropriate for your situation. There are three kinds of transpiler plugins:
- [**Transpiler stage plugin**](/api/qiskit/transpiler_plugins). Choose this if you are defining a pass manager that can be substituted for one of the [6 stages](transpiler-stages) of a preset staged pass manager.
- [**Unitary synthesis plugin**](/api/qiskit/qiskit.transpiler.passes.synthesis.plugin.UnitarySynthesisPlugin). Choose this if your transpilation code takes as input a unitary matrix and outputs a description of a quantum circuit implementing that unitary.
- [**High-level synthesis plugin**](/api/qiskit/qiskit.transpiler.passes.synthesis.plugin.HighLevelSynthesisPlugin). Choose this if your transpilation code takes as input a "high-level object" such as a Clifford operator or a linear function and outputs a description of a quantum circuit implementing that high-level object. High-level objects are represented by subclasses of the [Operation](/api/qiskit/qiskit.circuit.Operation) class.

Once you've determined which kind of plugin to create, follow these steps to create the plugin:

1. Create a subclass of the appropriate abstract plugin class:
   - [PassManagerStagePlugin](/api/qiskit/qiskit.transpiler.preset_passmanagers.plugin.PassManagerStagePlugin) for a transpiler stage plugin,
   - [UnitarySynthesisPlugin](/api/qiskit/qiskit.transpiler.passes.synthesis.plugin.UnitarySynthesisPlugin) for a unitary synthesis plugin, and
   - [HighLevelSynthesisPlugin](/api/qiskit/qiskit.transpiler.passes.synthesis.plugin.HighLevelSynthesisPlugin)  for a high-level synthesis plugin.
2. Expose the class as a [setuptools entry point](https://setuptools.pypa.io/en/latest/userguide/entry_point.html) in the package metadata, typically by adding an `entry-points` table in `pyproject.toml`.

There is no limit to the number of plugins a single package can define, but each plugin must have a unique name. The name `default` is used by Qiskit itself and can't be used in a plugin.

In the next sections, we show examples of these steps for the different types of plugins. In these examples, we assume that we are creating a Python package called `my_qiskit_plugin`, and that we are using a `pyproject.toml` file to store our project metadata. For information on creating Python packages, you can check out [this tutorial](https://packaging.python.org/en/latest/tutorials/packaging-projects/) from the Python website.

## Example: Create a transpiler stage plugin

In this example, we create a transpiler stage plugin for the `layout` stage (see [Transpiler stages](transpiler-stages) for a description of the 6 stages of Qiskit's built-in transpilation pipeline).
Our plugin simply runs [VF2Layout](/api/qiskit/qiskit.transpiler.passes.VF2Layout) for a number of trials that depends on the requested optimization level.

First, we create a subclass of [PassManagerStagePlugin](/api/qiskit/qiskit.transpiler.preset_passmanagers.plugin.PassManagerStagePlugin). There is one method we need to implement, called [`pass_manager`](/api/qiskit/qiskit.transpiler.preset_passmanagers.plugin.PassManagerStagePlugin#pass_manager). This method takes as input a [PassManagerConfig](/api/qiskit/qiskit.transpiler.PassManagerConfig) and returns the pass manager that we are defining. The PassManagerConfig object stores information about the target backend, such as its coupling map and basis gates.

In [1]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import VF2Layout
from qiskit.transpiler.passmanager_config import PassManagerConfig
from qiskit.transpiler.preset_passmanagers import common
from qiskit.transpiler.preset_passmanagers.plugin import PassManagerStagePlugin


class MyLayoutPlugin(PassManagerStagePlugin):
    def pass_manager(
        self,
        pass_manager_config: PassManagerConfig,
        optimization_level: int | None = None,
    ) -> PassManager:
        layout_pm = PassManager(
            [
                VF2Layout(
                    coupling_map=pass_manager_config.coupling_map,
                    properties=pass_manager_config.backend_properties,
                    max_trials=optimization_level * 10 + 1,
                    target=pass_manager_config.target,
                )
            ]
        )
        layout_pm += common.generate_embed_passmanager(pass_manager_config.coupling_map)
        return layout_pm

Now, we expose the plugin by adding an `entry-points` table in `pyproject.toml` under the appropriate namespace. See the [table of transpiler plugin stages](/api/qiskit/transpiler_plugins#stage-table) for the entry points and expectations for each stage.
To expose this plugin in a package called `my_qiskit_plugin`, we would add the following in `pyproject.toml`:

```toml
[project.entry-points."qiskit.transpiler.layout"]
"vf2" = "my_qiskit_plugin.module.plugin:MyLayoutPlugin"
```

If your project uses `setup.cfg` or `setup.py` instead of `pyproject.toml`, see the [setuptools documentation](https://setuptools.pypa.io/en/latest/userguide/entry_point.html) for how to adapt these lines for your situation.

## Example: Create a unitary synthesis plugin

In this example, we'll create a unitary synthesis plugin that simply uses the built-in [UnitarySynthesis](/api/qiskit/qiskit.transpiler.passes.UnitarySynthesis#unitarysynthesis) transpilation pass to synthesize a gate. Of course, your own plugin will do something more interesting than that.

The [UnitarySynthesisPlugin](/api/qiskit/qiskit.transpiler.passes.synthesis.plugin.UnitarySynthesisPlugin) class defines the interface and contract for unitary synthesis
plugins. The primary method is
[`run`](/api/qiskit/qiskit.transpiler.passes.synthesis.plugin.UnitarySynthesisPlugin#run),
which takes as input a Numpy array storing a unitary matrix
and returns a [DAGCircuit](/api/qiskit/qiskit.dagcircuit.DAGCircuit) representing the circuit synthesized from that unitary matrix.
In addition to the `run` method, there are a number of property methods that need to be defined.
See [UnitarySynthesisPlugin](/api/qiskit/qiskit.transpiler.passes.synthesis.plugin.UnitarySynthesisPlugin) for documentation of all required properties.

Let's create our UnitarySynthesisPlugin subclass:

In [2]:
import numpy as np
from qiskit.circuit import QuantumCircuit, QuantumRegister
from qiskit.converters import circuit_to_dag
from qiskit.dagcircuit.dagcircuit import DAGCircuit
from qiskit.quantum_info import Operator
from qiskit.transpiler.passes import UnitarySynthesis
from qiskit.transpiler.passes.synthesis.plugin import UnitarySynthesisPlugin


class MyUnitarySynthesisPlugin(UnitarySynthesisPlugin):
    @property
    def supports_basis_gates(self):
        return True

    @property
    def supports_coupling_map(self):
        return False

    @property
    def supports_natural_direction(self):
        return False

    @property
    def supports_pulse_optimize(self):
        return False

    @property
    def supports_gate_lengths(self):
        return False

    @property
    def supports_gate_errors(self):
        return False

    @property
    def supports_gate_lengths_by_qubit(self):
        return False

    @property
    def supports_gate_errors_by_qubit(self):
        return False

    @property
    def min_qubits(self):
        return None

    @property
    def max_qubits(self):
        return None

    @property
    def supported_bases(self):
        return None

    def run(self, unitary: np.ndarray, **options) -> DAGCircuit:
        basis_gates = options["basis_gates"]
        synth_pass = UnitarySynthesis(basis_gates, min_qubits=3)
        qubits = QuantumRegister(3)
        circuit = QuantumCircuit(qubits)
        circuit.append(Operator(unitary).to_instruction(), qubits)
        dag_circuit = synth_pass.run(circuit_to_dag(circuit))
        return dag_circuit

If you find that the inputs available to the [`run`](/api/qiskit/qiskit.transpiler.passes.synthesis.plugin.UnitarySynthesisPlugin#run)
method are insufficient for your purposes, please [open an issue](https://github.com/Qiskit/qiskit/issues/new/choose) explaining your requirements. Changes to the plugin interface, such as adding additional optional inputs, will be done in a backward compatible way so that they do not require changes from existing plugins.

<Admonition type="note" title="Note">
All methods prefixed with `supports_` are reserved on a `UnitarySynthesisPlugin` derived class as part of the interface. You should not define any custom `supports_*` methods on a subclass that are not defined in the abstract class.
</Admonition>

Now, we expose the plugin by adding an `entry-points` table in `pyproject.toml` under the `qiskit.unitary_synthesis` namespace.
To expose this plugin in a package called `my_qiskit_plugin`, we would add the following in `pyproject.toml`:

```toml
[project.entry-points."qiskit.unitary_synthesis"]
"special" = "my_qiskit_plugin.module.plugin:MyUnitarySynthesisPlugin"
```

As before, if your project uses `setup.cfg` or `setup.py` instead of `pyproject.toml`, see the [setuptools documentation](https://setuptools.pypa.io/en/latest/userguide/entry_point.html) for how to adapt these lines for your situation.

To accommodate unitary synthesis plugins that expose multiple options,
the plugin interface has an option for users to provide a free-form
configuration dictionary. This will be passed to the `run` method
via the `options` keyword argument. If your plugin has these configuration options, you should clearly document them.

## Example: Create a high-level synthesis plugin

In this example, we'll create a high-level synthesis plugin that simply uses the built-in [synth_clifford_bm](/api/qiskit/synthesis#synth_clifford_bm) function to synthesize a Clifford operator.

The [HighLevelSynthesisPlugin](/api/qiskit/qiskit.transpiler.passes.synthesis.plugin.HighLevelSynthesisPlugin) class defines the interface and contract for high-level synthesis plugins. The primary method is [`run`](/api/qiskit/qiskit.transpiler.passes.synthesis.plugin.HighLevelSynthesisPlugin#run).
The positional argument `high_level_object` is an [Operation](/api/qiskit/qiskit.circuit.Operation) representing the "high-level" object to be synthesized. For example, it could be a
[LinearFunction](/api/qiskit/qiskit.circuit.library.LinearFunction) or a
[Clifford](/api/qiskit/qiskit.quantum_info.Clifford).
The following keyword arguments are present:
- `target` specifies the target backend, allowing the plugin
to access all target-specific information,
such as the coupling map, the supported gate set, and so on
- `coupling_map` only specifies the coupling map, and is only used when `target` is not specified.
- `qubits` specifies the list of qubits over which the
high-level object is defined, in case the synthesis is done on the physical circuit.
A value of ``None`` indicates that the layout has not yet been chosen and the physical qubits in the target or coupling map that this operation is operating on has not yet been determined.
- `options`, a free-form configuration dictionary for plugin-specific options. If your plugin has these configuration options you
should clearly document them.

The `run` method returns a [QuantumCircuit](/api/qiskit/qiskit.circuit.QuantumCircuit)
representing the circuit synthesized from that high-level object.
It is also allowed to return `None`, indicating that the plugin is unable to synthesize the given high-level object.
The actual synthesis of high-level objects is performed by the
[HighLevelSynthesis](/api/qiskit/qiskit.transpiler.passes.HighLevelSynthesis)
transpiler pass.

In addition to the `run` method, there are a number of property methods that need to be defined.
See [HighLevelSynthesisPlugin](/api/qiskit/qiskit.transpiler.passes.synthesis.plugin.HighLevelSynthesisPlugin) for documentation of all required properties.

Let's define our HighLevelSynthesisPlugin subclass:

In [3]:
from qiskit.synthesis import synth_clifford_bm
from qiskit.transpiler.passes.synthesis.plugin import HighLevelSynthesisPlugin


class MyCliffordSynthesisPlugin(HighLevelSynthesisPlugin):
    def run(
        self, high_level_object, coupling_map=None, target=None, qubits=None, **options
    ) -> QuantumCircuit:
        if high_level_object.num_qubits <= 3:
            return synth_clifford_bm(high_level_object)
        else:
            return None

This plugin synthesizes objects of type [Clifford](/api/qiskit/qiskit.quantum_info.Clifford) that have
at most 3 qubits, using the `synth_clifford_bm` method.

Now, we expose the plugin by adding an `entry-points` table in `pyproject.toml` under the `qiskit.synthesis` namespace.
To expose this plugin in a package called `my_qiskit_plugin`, we would add the following in `pyproject.toml`:

```toml
[project.entry-points."qiskit.synthesis"]
"clifford.special" = "my_qiskit_plugin.module.plugin:MyCliffordSynthesisPlugin"
```

The `name` consists of two parts separated by a dot (`.`):
- The name of the type of [Operation](/api/qiskit/qiskit.circuit.Operation) that the plugin synthesizes (in this case, `clifford`). Note that this string corresponds to the [`name`](/api/qiskit/qiskit.circuit.Operation#name) attribute of the Operation class, and not the name of the class itself.
- The name of the plugin (in this case, `special`).

As before, if your project uses `setup.cfg` or `setup.py` instead of `pyproject.toml`, see the [setuptools documentation](https://setuptools.pypa.io/en/latest/userguide/entry_point.html) for how to adapt these lines for your situation.